In [1]:
import pandas as pd
import os
from enum import Enum

metadata_file = "./results/metadata.pkl"
dataset_types =  Enum("dataset_types", "train development test")
# Added to allow sorting
dataset_types.__lt__ = lambda self, other: self.value < other.value

def save_results(y_pred, index, name, task, lenguage, dataset_type, group=None, description=None, truth=False, filename=None):
    
    path = f"./results/{task}/{lenguage}/{dataset_type.name}{'/' + group if group is not None else ''}/{name if filename is None else filename}.pkl"
    
    directory = "/".join(path.split("/")[:-1])
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    if os.path.exists(metadata_file):
        metadata = pd.read_pickle(metadata_file)
    else:
        metadata = pd.DataFrame({
            "Path": pd.Series([], dtype=str),
            "Name": pd.Series([], dtype=str),
            "Description": pd.Series([], dtype=str),
            "Dataset type": pd.Categorical([], categories=dataset_types, ordered=False),
            "Groud Truth": pd.Series([], dtype=bool),
            "Group": pd.Series([], dtype=str),
            "Task": pd.Series([], dtype=str),
            "Lenguage": pd.Series([], dtype=str),
        }).set_index("Path")
    
    if path in metadata.index:
        metadata = remove_results(path)

    metadata.loc[path] = {"Name": name, "Description": description, "Dataset type": dataset_type, "Groud Truth": truth, "Group": group, "Task": task, "Lenguage": lenguage}
    results = pd.DataFrame({"id": index, "y_pred": y_pred}).set_index("id") 
    
    results.to_pickle(path)
    metadata.to_pickle(metadata_file)
    
    print("Results saved on: " + path)

def remove_results(path=None):
    if os.path.exists(metadata_file):
        metadata = pd.read_pickle(metadata_file)
        if path is not None:
            if os.path.exists(path):
                metadata = metadata.drop(path)
                os.remove(path)
                metadata.to_pickle(metadata_file)
        else:
            if os.path.exists(metadata_file):
                used_files = [os.path.normpath(f) for f in metadata.index]
                all_files = set([os.path.normpath(os.path.join(dp, f)) for dp, dn, filenames in os.walk('./results') for f in filenames][1:])
                for f in used_files:
                    if f not in all_files:
                        os.remove(f)
        return metadata

    
def load_results():
    if os.path.exists(metadata_file):
        metadata = pd.read_pickle(metadata_file)
    else:
        return None
    
    metadata["Results"] = [pd.read_pickle(path) for path in metadata.index]
    return metadata

In [2]:
from sklearn.metrics import accuracy_score, classification_report, f1_score

def print_score(y_true, y_pred, name):
    classification_report_results = classification_report(y_true, y_pred)
    
    acc, f1 = accuracy_score(y_true, y_pred), f1_score(y_true, y_pred, average='macro')

    print(name)
    print('F1 macro: ', f1)
    print('Accuracy: ', acc)

    print('\nClassification Report')
    print('======================================================')
    print('\n', classification_report_results)
    
    return {"F1 macro": f1, "Accuracy": acc}

In [3]:
df_results = load_results()

mask = df_results["Groud Truth"] == True

df_truth = df_results[mask]
df_pred = df_results[~mask]

df_truth.sort_values(by=["Lenguage", "Dataset type"])

,Name,Description,Dataset type,Groud Truth,Group,Task,Lenguage,Results
Path,,,,,,,,
./results/hateval2019/task1/english/train/train_truth_task1.pkl,English Train,None,dataset_types.train,True,None,hateval2019/task1,english,y_pred id 201 1 202 ...
./results/hateval2019/task1/english/development/dev_truth_task1.pkl,English Development,None,dataset_types.development,True,None,hateval2019/task1,english,y_pred id 18201 0 1820...
./results/hateval2019/task1/english/test/test_truth_task1.pkl,English Test,None,dataset_types.test,True,None,hateval2019/task1,english,y_pred id 34243 0 3059...
./results/hateval2019/task1/spanish/train/train_truth_task1.pkl,Spanish Train,None,dataset_types.train,True,None,hateval2019/task1,spanish,y_pred id 20001 1 2000...
./results/hateval2019/task1/spanish/development/dev_truth_task1.pkl,Spanish Development,None,dataset_types.development,True,None,hateval2019/task1,spanish,y_pred id 20005 0 2000...
./results/hateval2019/task1/spanish/test/test_truth_task1.pkl,Spanish Test,None,dataset_types.test,True,None,hateval2019/task1,spanish,y_pred id 31494 0 3246...


In [4]:
results = []

for path, (name, desc, dataset_type, truth, group, task, lenguage, y_pred) in df_pred.iterrows():
    y_true = df_truth[(df_truth["Dataset type"] == dataset_type) & (df_truth["Task"] == task) & (df_truth["Lenguage"] == lenguage)]["Results"][0]
    
    result = print_score(y_true, y_pred, name)
    result.update({"Dataset type": dataset_type, "Name": name, "Group": group, "Lenguage": lenguage, "Description": desc})
    
    results.append(result)

Bert base
F1 macro:  0.9299943995519642
Accuracy:  0.9315555555555556

Classification Report

               precision    recall  f1-score   support

           0       0.95      0.93      0.94      5217
           1       0.91      0.93      0.92      3783

    accuracy                           0.93      9000
   macro avg       0.93      0.93      0.93      9000
weighted avg       0.93      0.93      0.93      9000

Bert base
F1 macro:  0.7469041129594169
Accuracy:  0.749

Classification Report

               precision    recall  f1-score   support

           0       0.81      0.73      0.77       573
           1       0.68      0.77      0.72       427

    accuracy                           0.75      1000
   macro avg       0.75      0.75      0.75      1000
weighted avg       0.76      0.75      0.75      1000

Bert base
F1 macro:  0.5941497231031642
Accuracy:  0.6053333333333333

Classification Report

               precision    recall  f1-score   support

           0       

C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best Ridge Classifier
F1 macro:  0.8271567865321129
Accuracy:  0.8356666666666667

Classification Report

               precision    recall  f1-score   support

           0       0.82      0.91      0.87      5217
           1       0.86      0.73      0.79      3783

    accuracy                           0.84      9000
   macro avg       0.84      0.82      0.83      9000
weighted avg       0.84      0.84      0.83      9000

Random Forest classifier
F1 macro:  0.9989737648560049
Accuracy:  0.999

Classification Report

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5217
           1       1.00      1.00      1.00      3783

    accuracy                           1.00      9000
   macro avg       1.00      1.00      1.00      9000
weighted avg       1.00      1.00      1.00      9000

Best Random Forest classifier
F1 macro:  0.9965763219746517
Accuracy:  0.9966666666666667

Classification Report

               precision    

C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\

Best Support Vector Classification
F1 macro:  0.7069521297684922
Accuracy:  0.728

Classification Report

               precision    recall  f1-score   support

           0       0.72      0.87      0.79       573
           1       0.75      0.54      0.63       427

    accuracy                           0.73      1000
   macro avg       0.74      0.70      0.71      1000
weighted avg       0.73      0.73      0.72      1000

AdaBoost classifier
F1 macro:  0.7258515670051333
Accuracy:  0.739

Classification Report

               precision    recall  f1-score   support

           0       0.74      0.84      0.79       573
           1       0.73      0.61      0.67       427

    accuracy                           0.74      1000
   macro avg       0.74      0.72      0.73      1000
weighted avg       0.74      0.74      0.73      1000

Best AdaBoost classifier
F1 macro:  0.7046539875871478
Accuracy:  0.716

Classification Report

               precision    recall  f1-score   supp

C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\

Bernoulli Naive Bayes classifier
F1 macro:  0.8776877260140155
Accuracy:  0.8808888888888889

Classification Report

               precision    recall  f1-score   support

           0       0.91      0.89      0.90      2643
           1       0.84      0.87      0.86      1857

    accuracy                           0.88      4500
   macro avg       0.88      0.88      0.88      4500
weighted avg       0.88      0.88      0.88      4500

Best Bernoulli Naive Bayes classifier
F1 macro:  0.871494135645079
Accuracy:  0.8755555555555555

Classification Report

               precision    recall  f1-score   support

           0       0.89      0.90      0.89      2643
           1       0.85      0.85      0.85      1857

    accuracy                           0.88      4500
   macro avg       0.87      0.87      0.87      4500
weighted avg       0.88      0.88      0.88      4500

Ridge Classifier
F1 macro:  0.9337121832097052
Accuracy:  0.936

Classification Report

               pre

Best Support Vector Classification
F1 macro:  0.7173983389062669
Accuracy:  0.738

Classification Report

               precision    recall  f1-score   support

           0       0.71      0.91      0.79       278
           1       0.82      0.53      0.64       222

    accuracy                           0.74       500
   macro avg       0.76      0.72      0.72       500
weighted avg       0.76      0.74      0.73       500

AdaBoost classifier
F1 macro:  0.741819819149947
Accuracy:  0.75

Classification Report

               precision    recall  f1-score   support

           0       0.75      0.83      0.79       278
           1       0.76      0.64      0.70       222

    accuracy                           0.75       500
   macro avg       0.75      0.74      0.74       500
weighted avg       0.75      0.75      0.75       500

Best AdaBoost classifier
F1 macro:  0.74581413273422
Accuracy:  0.752

Classification Report

               precision    recall  f1-score   support


C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ferran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Bert Avarage
F1 macro:  0.5939885574846252
Accuracy:  0.606

Classification Report

               precision    recall  f1-score   support

           0       0.88      0.37      0.52      1740
           1       0.52      0.93      0.66      1260

    accuracy                           0.61      3000
   macro avg       0.70      0.65      0.59      3000
weighted avg       0.72      0.61      0.58      3000

Bert Avarage
F1 macro:  0.9835327664403473
Accuracy:  0.984

Classification Report

               precision    recall  f1-score   support

           0       0.99      0.98      0.99      2643
           1       0.97      0.99      0.98      1857

    accuracy                           0.98      4500
   macro avg       0.98      0.98      0.98      4500
weighted avg       0.98      0.98      0.98      4500

Bert Avarage
F1 macro:  0.8312195592419347
Accuracy:  0.832

Classification Report

               precision    recall  f1-score   support

           0       0.88      0.81   

In [5]:
df_results = pd.DataFrame(results).set_index(["Lenguage", "Dataset type", "Group", "Name"]).groupby(level=[0, 1, 2, 3]).sum()
df_results_index = df_results.sort_values(by=["Lenguage", "Dataset type", "F1 macro", "Accuracy"], ascending=[True, True, False, False]).droplevel("Group")
df_results_index

F1 macro  \
Lenguage Dataset type        Name                                                
english  dataset_types.train Random Forest classifier                 0.998974   
                             Best Random Forest classifier            0.996576   
                             Support Vector Classification            0.956309   
                             Entire layer                             0.940645   
                             Bert Avarage                             0.939383   
...                                                                        ...   
spanish  dataset_types.test  Best Support Vector Classification       0.682732   
                             Multinomial Naive Bayes classifier       0.670608   
                             AdaBoost classifier                      0.659715   
                             Best Multinomial Naive Bayes classifier  0.655271   
                             Dummy Classifier                         0.370079   

                                                                      Accuracy  
Lenguage Dataset type        Name                                               
english  dataset_types.train Random Forest classifier                 0.999000  
                             Best Random Forest classifier            0.996667  
                             Support Vector Classification            0.957556  
                             Entire layer                             0.942000  
                             Bert Avarage                             0.940778  
...                                                                        ...  
spanish  dataset_types.test  Best Support Vector Classification       0.725000  
                             Multinomial Naive Bayes classifier       0.700625  
                             AdaBoost classifier                      0.667500  
                             Best Multinomial Naive Bayes classifier  0.697500  
                             Dummy Classifier                         0.587500  

[120 rows x 2 columns]

In [6]:
a = [lambda x: 1, lambda x: 2, lambda x: 3] # Objective
b = [(lambda x: i+1) for i in range(3)] # Problem
c = [(lambda i: lambda x: i+1)(i) for i in range(3)] #First solution

print(a[0](1), b[0](1), c[0](1))

1 3 1


In [7]:
list_aggregate = [(lambda t: lambda x: x[:, t])(t) for t in dataset_types]
df_results_columns = df_results.groupby(["Lenguage",'Group','Name']).aggregate(list_aggregate)
df_results_columns.columns = pd.MultiIndex.from_product([['F1 macro', 'Accuracy'], [t.name.capitalize() for t in dataset_types]])
df_results_columns = df_results_columns.sort_values(by=['Lenguage', 'Group', ("F1 macro", dataset_types.test.name.capitalize())], ascending=[True, False, False])
df_results_columns.index = df_results_columns.index.set_levels([' '.join(i.split('_')).title() for i in df_results_columns.index.levels[0]], level=0)

df_results_columns

F1 macro  \
                                                                      Train   
Lenguage Group            Name                                                
English  traditional      Ridge Classifier                         0.877431   
                          Multi-layer Perceptron classifier        0.888015   
                          Multinomial Naive Bayes classifier       0.812630   
                          Support Vector Classification            0.956309   
                          AdaBoost classifier                      0.767939   
                          Bernoulli Naive Bayes classifier         0.821029   
                          Random Forest classifier                 0.998974   
                          Dummy Classifier                         0.366955   
         deep_learning    Bert base                                0.929994   
                          Bert Avarage                             0.939383   
                          Atalaya                                  0.901578   
                          GPT2 base                                0.805459   
         best_traditional Best Multi-layer Perceptron classifier   0.817371   
                          Best Multinomial Naive Bayes classifier  0.807131   
                          Best Ridge Classifier                    0.827157   
                          Best Support Vector Classification       0.807855   
                          Best AdaBoost classifier                 0.814896   
                          Best Bernoulli Naive Bayes classifier    0.815134   
                          Best Random Forest classifier            0.996576   
         bert_avarage     CLS layers                               0.931599   
                          Entire layer                             0.940645   
                          Tokens                                   0.936114   
Spanish  traditional      Support Vector Classification            0.976335   
                          Multi-layer Perceptron classifier        0.912374   
                          Bernoulli Naive Bayes classifier         0.877688   
                          Random Forest classifier                 0.999771   
                          Ridge Classifier                         0.933712   
                          Multinomial Naive Bayes classifier       0.851474   
                          AdaBoost classifier                      0.769740   
                          Dummy Classifier                         0.370013   
         deep_learning    Bert Avarage                             0.983533   
                          Bert base                                0.959198   
                          Atalaya                                  0.946563   
         best_traditional Best Ridge Classifier                    0.863485   
                          Best Random Forest classifier            0.999542   
                          Best Bernoulli Naive Bayes classifier    0.871494   
                          Best Multi-layer Perceptron classifier   0.869665   
                          Best AdaBoost classifier                 0.863730   
                          Best Support Vector Classification       0.811682   
                          Best Multinomial Naive Bayes classifier  0.819576   

                                                                               \
                                                                  Development   
Lenguage Group            Name                                                  
English  traditional      Ridge Classifier                           0.709848   
                          Multi-layer Perceptron classifier          0.693095   
                          Multinomial Naive Bayes classifier         0.730203   
                          Support Vector Classification              0.734485   
                          AdaBoost classifier                        0.725852   
                     

In [8]:
df_results_columns2 = df_results_columns.droplevel('Group')
df_results_columns2 = df_results_columns2.sort_values(by=["Lenguage", ("F1 macro", dataset_types.test.name.capitalize())], ascending=[True, False])
df_results_columns2

F1 macro              \
                                                     Train Development   
Lenguage Name                                                            
English  Bert base                                0.929994    0.746904   
         Bert Avarage                             0.939383    0.750845   
         CLS layers                               0.931599    0.750351   
         Entire layer                             0.940645    0.757246   
         Tokens                                   0.936114    0.748223   
         Best Multi-layer Perceptron classifier   0.817371    0.732399   
         Ridge Classifier                         0.877431    0.709848   
         Best Multinomial Naive Bayes classifier  0.807131    0.732959   
         Multi-layer Perceptron classifier        0.888015    0.693095   
         Best Ridge Classifier                    0.827157    0.733554   
         Multinomial Naive Bayes classifier       0.812630    0.730203   
         Atalaya                                  0.901578    0.740024   
         Support Vector Classification            0.956309    0.734485   
         Best Support Vector Classification       0.807855    0.706952   
         Best AdaBoost classifier                 0.814896    0.704654   
         GPT2 base                                0.805459    0.735624   
         Best Bernoulli Naive Bayes classifier    0.815134    0.734426   
         AdaBoost classifier                      0.767939    0.725852   
         Bernoulli Naive Bayes classifier         0.821029    0.734550   
         Random Forest classifier                 0.998974    0.731493   
         Best Random Forest classifier            0.996576    0.750622   
         Dummy Classifier                         0.366955    0.364272   
Spanish  Bert Avarage                             0.983533    0.831220   
         Bert base                                0.959198    0.833183   
         Best Ridge Classifier                    0.863485    0.735636   
         Best Random Forest classifier            0.999542    0.725951   
         Support Vector Classification            0.976335    0.741667   
         Atalaya                                  0.946563    0.739700   
         Multi-layer Perceptron classifier        0.912374    0.729136   
         Bernoulli Naive Bayes classifier         0.877688    0.761420   
         Random Forest classifier                 0.999771    0.728113   
         Ridge Classifier                         0.933712    0.733851   
         Best Bernoulli Naive Bayes classifier    0.871494    0.766972   
         Best Multi-layer Perceptron classifier   0.869665    0.699844   
         Best AdaBoost classifier                 0.863730    0.745814   
         Best Support Vector Classification       0.811682    0.717398   
         Multinomial Naive Bayes classifier       0.851474    0.709441   
         AdaBoost classifier                      0.769740    0.741820   
         Best Multinomial Naive Bayes classifier  0.819576    0.689804   
         Dummy Classifier                         0.370013    0.357326   

                                                            Accuracy  \
                                                      Test     Train   
Lenguage Name                                                          
English  Bert base                                0.594150  0.931556   
         Bert Avarage                             0.593989  0.940778   
         CLS layers                               0.589766  0.933111   
         Entire layer                             0.586625  0.942000   
         Tokens                                   0.586530  0.937444   
         Best Multi-layer Perceptron classifier   0.488366  0.821778   
         Ridge Classifier                         0.487733  0.881667   
         Best Multinomial Naive Bayes classifier  0.483788  0.816222   
         Multi-layer Perceptron classifier        0.483704  0.891444   
   

Improvements:
- Allow removal of all files with specific features (not just path)